# SSI - predicting CS:GO winning team
This is an example of using Machine Learning in case of data from CS:GO proffesional games from 2019-2020. Here we try to predict which team (CT or TT) will win round based on weapons and status information of each team.  

The data has been mined from demos by others (using external programs) and can be obtained from site [1].

Following Notebook present all steps that has been done in this project. As the matter of fact, it contains:
   1. Importing modules, such as pandas, sklearn
   2. Loading data (over 120 000 records)
   3. Pre-processing data, including:
       1. Checking for NaN values
       2. Checking for single-calued volumns
       3. Dealing with non-numerical data
       4. Normalizing data
   4. Performing multiple Supervised Learning and comparing results

In [1]:
import pandas as pd
import numpy as np
from time import time

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report

pd.set_option('display.max_columns', 500)

## Here we load data

In [2]:
df = pd.read_csv('data/csgo_round_snapshots.csv')
df

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,t_weapon_aug,ct_weapon_awp,t_weapon_awp,ct_weapon_bizon,t_weapon_bizon,ct_weapon_cz75auto,t_weapon_cz75auto,ct_weapon_elite,t_weapon_elite,ct_weapon_famas,t_weapon_famas,ct_weapon_g3sg1,t_weapon_g3sg1,ct_weapon_galilar,t_weapon_galilar,ct_weapon_glock,t_weapon_glock,ct_weapon_m249,t_weapon_m249,ct_weapon_m4a1s,t_weapon_m4a1s,ct_weapon_m4a4,t_weapon_m4a4,ct_weapon_mac10,t_weapon_mac10,ct_weapon_mag7,t_weapon_mag7,ct_weapon_mp5sd,t_weapon_mp5sd,ct_weapon_mp7,t_weapon_mp7,ct_weapon_mp9,t_weapon_mp9,ct_weapon_negev,t_weapon_negev,ct_weapon_nova,t_weapon_nova,ct_weapon_p90,t_weapon_p90,ct_weapon_r8revolver,t_weapon_r8revolver,ct_weapon_sawedoff,t_weapon_sawedoff,ct_weapon_scar20,t_weapon_scar20,ct_weapon_sg553,t_weapon_sg553,ct_weapon_ssg08,t_weapon_ssg08,ct_weapon_ump45,t_weapon_ump45,ct_weapon_xm1014,t_weapon_xm1014,ct_weapon_deagle,t_weapon_deagle,ct_weapon_fiveseven,t_weapon_fiveseven,ct_weapon_usps,t_weapon_usps,ct_weapon_p250,t_weapon_p250,ct_weapon_p2000,t_weapon_p2000,ct_weapon_tec9,t_weapon_tec9,ct_grenade_hegrenade,t_grenade_hegrenade,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,de_train,T

## Here we prepare dataset as follow

    1. checking for NaN values
    2. removing columns with only one value
    3. casting categorical and boolean values to numeric

In [3]:
df_prepared = df.copy()
# checking on mising values, after this we choose how to handle with them
print(f'There are {df_prepared.isna().sum().sum()} missing values.')

# remove columsn with only one value
df_prepared = df_prepared.loc[:,df_prepared.nunique()!=1]

# cast categorical and boolean values to numeric
label_encoder = preprocessing.LabelEncoder() 
df_prepared.map = label_encoder.fit_transform(df_prepared.map)
df_prepared.bomb_planted = label_encoder.fit_transform(df_prepared.bomb_planted)
df_prepared.round_winner = label_encoder.fit_transform(df_prepared.round_winner)

#
df_prepared

There are 0 missing values.


,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,t_weapon_aug,ct_weapon_awp,t_weapon_awp,t_weapon_bizon,ct_weapon_cz75auto,t_weapon_cz75auto,ct_weapon_elite,t_weapon_elite,ct_weapon_famas,t_weapon_famas,t_weapon_g3sg1,ct_weapon_galilar,t_weapon_galilar,ct_weapon_glock,t_weapon_glock,ct_weapon_m249,ct_weapon_m4a1s,t_weapon_m4a1s,ct_weapon_m4a4,t_weapon_m4a4,ct_weapon_mac10,t_weapon_mac10,ct_weapon_mag7,t_weapon_mag7,ct_weapon_mp5sd,t_weapon_mp5sd,ct_weapon_mp7,t_weapon_mp7,ct_weapon_mp9,t_weapon_mp9,t_weapon_negev,ct_weapon_nova,t_weapon_nova,ct_weapon_p90,t_weapon_p90,t_weapon_r8revolver,t_weapon_sawedoff,ct_weapon_scar20,t_weapon_scar20,ct_weapon_sg553,t_weapon_sg553,ct_weapon_ssg08,t_weapon_ssg08,ct_weapon_ump45,t_weapon_ump45,ct_weapon_xm1014,t_weapon_xm1014,ct_weapon_deagle,t_weapon_deagle,ct_weapon_fiveseven,t_weapon_fiveseven,ct_weapon_usps,t_weapon_usps,ct_weapon_p250,t_weapon_p250,ct_weapon_p2000,t_weapon_p2000,ct_weapon_tec9,t_weapon_tec9,ct_grenade_hegrenade,t_grenade_hegrenade,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,1,0,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,156.03,0.0,0.0,1,0,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,96.03,0.0,0.0,1,0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,76.03,0.0,0.0,1,0,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,174.97,1.0,0.0,1,0,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,6,1,200.0,242.0,195.0,359.0,100.0,5950.0,2.0,4.0,1.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0

# Normalization
Here we normalize data using z-score as follows

$z = \frac{x-\mu}{\sigma},$

where $\mu$ is a mean of $x$ values and $\sigma$ is standard deviation of $x$.

In [4]:
standscl = preprocessing.StandardScaler()
standscl.fit(df_prepared)

# Here we are using z-score
df_scaled = pd.DataFrame(standscl.transform(df_prepared), columns=df_prepared.columns)
# except for the round_winner columns as this is what we are going to predict
df_scaled.round_winner = label_encoder.fit_transform(df_scaled.round_winner)

df_scaled

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,t_weapon_aug,ct_weapon_awp,t_weapon_awp,t_weapon_bizon,ct_weapon_cz75auto,t_weapon_cz75auto,ct_weapon_elite,t_weapon_elite,ct_weapon_famas,t_weapon_famas,t_weapon_g3sg1,ct_weapon_galilar,t_weapon_galilar,ct_weapon_glock,t_weapon_glock,ct_weapon_m249,ct_weapon_m4a1s,t_weapon_m4a1s,ct_weapon_m4a4,t_weapon_m4a4,ct_weapon_mac10,t_weapon_mac10,ct_weapon_mag7,t_weapon_mag7,ct_weapon_mp5sd,t_weapon_mp5sd,ct_weapon_mp7,t_weapon_mp7,ct_weapon_mp9,t_weapon_mp9,t_weapon_negev,ct_weapon_nova,t_weapon_nova,ct_weapon_p90,t_weapon_p90,t_weapon_r8revolver,t_weapon_sawedoff,ct_weapon_scar20,t_weapon_scar20,ct_weapon_sg553,t_weapon_sg553,ct_weapon_ssg08,t_weapon_ssg08,ct_weapon_ump45,t_weapon_ump45,ct_weapon_xm1014,t_weapon_xm1014,ct_weapon_deagle,t_weapon_deagle,ct_weapon_fiveseven,t_weapon_fiveseven,ct_weapon_usps,t_weapon_usps,ct_weapon_p250,t_weapon_p250,ct_weapon_p2000,t_weapon_p2000,ct_weapon_tec9,t_weapon_tec9,ct_grenade_hegrenade,t_grenade_hegrenade,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,1.415828,-1.400576,-1.405702,-1.306410,-0.354764,0.664386,0.695301,-1.836777,-1.709542,-0.516186,-0.595345,-1.115364,-1.379297,-1.004951,0.602448,0.597412,-0.470136,-0.912049,-0.292446,-0.083519,-0.852132,-0.673830,-0.009039,-0.340853,-0.279094,-0.063461,-0.042141,-0.283147,-0.066942,-0.021331,-0.098501,-0.271725,-0.079592,1.039382,-0.002858,-0.242763,-0.051389,-0.882806,-0.213485,-0.080226,-0.260603,-0.065298,-0.007001,-0.038268,-0.05062,-0.044438,-0.043684,-0.311208,-0.069177,-0.002858,-0.031325,-0.010306,-0.012783,-0.01246,-0.007562,-0.011785,-0.026822,-0.002858,-0.431618,-0.723924,-0.224726,-0.139955,-0.201795,-0.172652,-0.085777,-0.01617,-0.528063,-0.467479,-0.214046,-0.093784,0.516902,-0.278862,-0.44395,-0.503436,1.942778,-0.065959,-0.085586,-0.147871,-0.701566,-0.542299,-1.045338,-1.035461,-0.886648,-0.889569,-0.687185,-0.137694,-0.210881,-0.812929,-0.163356,-0.156855,0
1,1.067531,-1.400576,-1.405702,-1.306410,-0.354764,0.664386,0.695301,0.502008,0.008909,-0.819351,-0.870776,-1.115364,-1.379297,-0.382199,0.602448,0.597412,-0.470136,-0.912049,-0.292446,-0.083519,-0.852132,-0.673830,-0.009039,-0.340853,-0.279094,-0.063461,-0.042141,-0.283147,-0.066942,-0.021331,-0.098501,-0.271725,-0.079592,1.039382,-0.002858,-0.242763,-0.051389,-0.882806,-0.213485,-0.080226,-0.260603,-0.065298,-0.007001,-0.038268,-0.05062,-0.044438,-0.043684,-0.311208,-0.069177,-0.002858,-0.031325,-0.010306,-0.012783,-0.01246,-0.007562,-0.011785,-0.026822,-0.002858,-0.431618,-0.723924,-0.224726,-0.139955,-0.201795,-0.172652,-0.085777,-0.01617,-0.528063,-0.467479,-0.214046,-0.093784,0.516902,-0.278862,-0.44395,-0.503436,1.942778,-0.065959,-0.085586,-0.147871,-0.701566,-0.542299,-1.045338,-1.035461,-0.886648,0.203841,-0.687185,-0.137694,-0.210881,-0.812929,-0.163356,-0.156855,0
2,-0.034094,-1.400576,-1.405702,-1.306410,-0.354764,-0.159544,-0.019401,-0.117770,-0.563908,-0.805977,-0.883109,-1.115364,-1.379297,-0.382199,-0.227087,-0.216709,-0.470136,-0.912049,-0.292446,-0.083519,-0.852132,-0.673830,-0.009039,-0.340853,-0.279094,-0.063461,-0.042141,-0.283147,-0.066942,-0.021331,-0.098501,-0.271725,-0.079592,0.424292,-0.002858,-0.242763,-0.051389,-0.882806,-0.213485,-0.080226,-0.260603,-0.065298,-0.007001,-0.038268,-0.05062,-0.044438,-0.043684,-0.311208,-0.069177,-0.002858,-0.031325,-0.010306,-0.012783,-0.01246,-0.007562,-0.011785,-0.026822,-0.002858,-0.431618,-0.723924,-0.224726,-0.139955,-0.201795,-0.172652,-0.085777,-0.01617,-0.528063,-0.467479,-0.214046,-0.093784,0.516902,-0.278862,-0.44395,-0.503436,-0.409385,-0.065959,-0.085586,-0.147871,-0.701566,-0.542299,-1.

## Now we are off to training AI
First we split processed and normalized data set. For reproducity of this results we set fixed random_state value.  

In [5]:
data_train, data_test, results_train, results_test = train_test_split(df_scaled.drop('round_winner', axis=1), df_scaled.round_winner, 
                                                                      test_size=0.2, random_state=1)
data_train

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,t_weapon_aug,ct_weapon_awp,t_weapon_awp,t_weapon_bizon,ct_weapon_cz75auto,t_weapon_cz75auto,ct_weapon_elite,t_weapon_elite,ct_weapon_famas,t_weapon_famas,t_weapon_g3sg1,ct_weapon_galilar,t_weapon_galilar,ct_weapon_glock,t_weapon_glock,ct_weapon_m249,ct_weapon_m4a1s,t_weapon_m4a1s,ct_weapon_m4a4,t_weapon_m4a4,ct_weapon_mac10,t_weapon_mac10,ct_weapon_mag7,t_weapon_mag7,ct_weapon_mp5sd,t_weapon_mp5sd,ct_weapon_mp7,t_weapon_mp7,ct_weapon_mp9,t_weapon_mp9,t_weapon_negev,ct_weapon_nova,t_weapon_nova,ct_weapon_p90,t_weapon_p90,t_weapon_r8revolver,t_weapon_sawedoff,ct_weapon_scar20,t_weapon_scar20,ct_weapon_sg553,t_weapon_sg553,ct_weapon_ssg08,t_weapon_ssg08,ct_weapon_ump45,t_weapon_ump45,ct_weapon_xm1014,t_weapon_xm1014,ct_weapon_deagle,t_weapon_deagle,ct_weapon_fiveseven,t_weapon_fiveseven,ct_weapon_usps,t_weapon_usps,ct_weapon_p250,t_weapon_p250,ct_weapon_p2000,t_weapon_p2000,ct_weapon_tec9,t_weapon_tec9,ct_grenade_hegrenade,t_grenade_hegrenade,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade
107416,-0.420764,0.269451,-0.783750,-0.269647,-0.354764,-0.461904,-0.891337,0.455232,-0.151480,0.696476,-0.336358,0.513776,0.112567,0.240554,-0.227087,-1.030829,-0.470136,-0.178847,-0.292446,-0.083519,0.633208,1.424079,-0.009039,-0.340853,-0.279094,-0.063461,-0.042141,-0.283147,-0.066942,-0.021331,-0.098501,-0.271725,-0.079592,-0.190798,-0.002858,-0.242763,-0.051389,1.578010,-0.213485,-0.080226,-0.260603,-0.065298,-0.007001,-0.038268,-0.05062,-0.044438,-0.043684,-0.311208,-0.069177,-0.002858,-0.031325,-0.010306,-0.012783,-0.01246,-0.007562,-0.011785,-0.026822,-0.002858,-0.431618,0.153623,-0.224726,-0.139955,-0.201795,-0.172652,-0.085777,-0.01617,-0.528063,-0.467479,-0.214046,-0.093784,-0.113304,-0.278862,-0.443950,-0.503436,1.942778,-0.065959,-0.085586,-0.147871,0.859028,-0.542299,-0.481253,0.636346,-0.311206,-0.889569,-0.001350,-0.137694,-0.210881,-0.211693,-0.163356,-0.156855
114728,-0.936692,0.686957,1.496741,1.803880,-0.354764,-2.359211,-1.541716,-1.252080,-0.248858,-0.841643,0.222726,-1.115364,0.112567,-1.004951,-2.715690,-1.030829,-0.470136,1.287556,-0.292446,-0.083519,-0.852132,-0.673830,-0.009039,-0.340853,-0.279094,-0.063461,-0.042141,2.287692,-0.066942,-0.021331,-0.098501,-0.271725,-0.079592,-0.805888,-0.002858,-0.242763,-0.051389,-0.882806,-0.213485,-0.080226,-0.260603,-0.065298,-0.007001,-0.038268,-0.05062,-0.044438,-0.043684,-0.311208,-0.069177,-0.002858,-0.031325,-0.010306,-0.012783,-0.01246,-0.007562,-0.011785,-0.026822,-0.002858,-0.431618,-0.723924,-0.224726,-0.139955,-0.201795,-0.172652,-0.085777,-0.01617,-0.528063,-0.467479,-0.214046,-0.093784,-1.373717,-0.278862,-0.443950,1.222917,-0.409385,-0.065959,-0.085586,-0.147871,-0.701566,-0.542299,-0.481253,-0.478192,-0.311206,-0.889569,-0.687185,6.810000,-0.210881,-0.812929,-0.163356,-0.156855
65486,0.313285,-0.774316,-1.405702,0.248735,-0.354764,0.664386,0.695301,1.086704,1.154543,-0.663310,-0.874887,1.056823,1.107144,0.863307,0.602448,0.597412,-0.470136,-0.178847,-0.292446,-0.083519,-0.852132,-0.673830,-0.009039,-0.340853,-0.279094,-0.063461,-0.042141,-0.283147,-0.066942,-0.021331,-0.098501,-0.271725,-0.079592,1.039382,-0.002858,3.561209,-0.051389,0.757738,-0.213485,-0.080226,-0.260603,-0.065298,-0.007001,-0.038268,-0.05062,-0.044438,-0.043684,-0.311208,-0.069177,-0.002858,-0.031325,-0.010306,-0.012783,-0.01246,-0.007562,-0.011785,-0.026822,-0.002858,3.178028,1.908717,-0.224726,-0.139955,-0.201795,4.561738,-0.085777,-0.01617,0.670242,-0.467479,-0.214046,-0.093784,-0.113304,-0.278862,-0.443950,-0.503436,1.942778,-0.065959,-0.085586,-0.147871,0.859028,1.932503,1.775085,1.750884,1.41511

# Here we use multiple ML methods, such as:
    1. Linear Discriminant Analysis (LDA)
    2. Quadratic Discriminant Analysis (QDA)
    3. Naive Bayes classifier
    4. Logistic Regression
    5. Decision Tree
    6. Random Forest (RF)
    7. Artificial Neural Network

to teach AI how to predicts. Also we print popular metrics, such as precission and recall, so we can compare different ML methods.

In [6]:
def perform_ML(method, data_train, data_test, results_train, results_test):
    model = method()
    
    dt = time()
    model.fit(data_train, results_train)
    pred = model.predict(data_test)
    dt = time()-dt
    
    print(f'It took {dt:.2f} second to perform {method.__name__}.')
    print(classification_report(results_test, pred))
    print('--------------------------------------------------------\n')


In [7]:
methods = [LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis, GaussianNB,
           LogisticRegression, DecisionTreeClassifier, RandomForestClassifier,
           MLPClassifier
          ]

for method in methods:
    perform_ML(method, data_train, data_test, results_train, results_test)

It took 0.82 second to perform LinearDiscriminantAnalysis.
              precision    recall  f1-score   support

           0       0.73      0.78      0.76     12012
           1       0.78      0.73      0.75     12470

    accuracy                           0.75     24482
   macro avg       0.75      0.75      0.75     24482
weighted avg       0.76      0.75      0.75     24482

--------------------------------------------------------



C:\Users\przempol\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


It took 0.27 second to perform QuadraticDiscriminantAnalysis.
              precision    recall  f1-score   support

           0       0.74      0.08      0.15     12012
           1       0.52      0.97      0.68     12470

    accuracy                           0.54     24482
   macro avg       0.63      0.53      0.41     24482
weighted avg       0.63      0.54      0.42     24482

--------------------------------------------------------

It took 0.18 second to perform GaussianNB.
              precision    recall  f1-score   support

           0       0.78      0.43      0.56     12012
           1       0.62      0.88      0.73     12470

    accuracy                           0.66     24482
   macro avg       0.70      0.66      0.64     24482
weighted avg       0.70      0.66      0.64     24482

--------------------------------------------------------

It took 0.67 second to perform LogisticRegression.
              precision    recall  f1-score   support

           0       

C:\Users\przempol\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
